In [1]:
import sys 
sys.path.append("..") 
import network_builder
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
from networkx.algorithms import community


Bad key "text.kerning_factor" on line 4 in
/mounts/data/proj/yihong/newhome/ENTER/envs/concept-net/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# we first build a DIRECTED graph
# which is the base of other UNDIRECTED graphs to be constructed
considered_lang = 'all'
# load from disk
net = network_builder.ConceptNetwork(involved_lang=considered_lang, 
                                     load_directed_graph_from_path=True, use_updated=True,
                                     load_directed_graph_path= \
                                     '/mounts/data/proj/yihong/newhome/ConceptNetwork/network_related/stored_networks')

Loading directed net from /mounts/data/proj/yihong/newhome/ConceptNetwork/network_related/stored_networks/directed_network_updated.pickle ...


In [3]:
print(f"Number of nodes: {net.concept_net.number_of_nodes()}")
print(f"Number of edges: {net.concept_net.number_of_edges()}")

# Number of nodes: 5870
# Number of edges: 1161424

Number of nodes: 5870
Number of edges: 1161424


In [4]:
# the following is the test
undirected_net = net.to_undirected(aggregate_type='union', minimum_number_of_langs=50)
print(f"Number of nodes: {undirected_net.number_of_nodes()}")
print(f"Number of edges: {undirected_net.number_of_edges()}")

# Number of nodes: 2591
# Number of edges: 13607

Number of nodes: 2591
Number of edges: 13607


## create subgraphs for each language family

In [5]:
import pickle5

with open('../iso2family.pickle', 'rb') as handle:
    iso2family = pickle5.load(handle)

families = {}

for key, item in iso2family.items():
    if item[0] not in families:
        families[item[0]] = [key]
    else:
        families[item[0]].append(key)

In [6]:
subgraph_dict_language_families = {}
for code in ['sino1245', 'otom1299', 'nucl1709', 'indo1319', 'atla1278', 'aust1307']:
    print(code)
    # the threshold 0.1 here controls the minimum number of languages for an edge
    # in the original undirected_net to be added in the language family subgraph
    # the value should be between 0 to 1
    subgraph_dict_language_families[code] = net.create_subgraph_of_a_language_family(code, 0.1, 
                                                                                     family_path='../iso2family.pickle')
    print(f"Number of languages: {len(families[code])}")
    print(f"Number of nodes: {subgraph_dict_language_families[code].number_of_nodes()}")
    print(f"Number of edges: {subgraph_dict_language_families[code].number_of_edges()}")
    print()

sino1245
Number of languages: 69
Number of nodes: 2591
Number of edges: 7460

otom1299
Number of languages: 76
Number of nodes: 2591
Number of edges: 7290

nucl1709
Number of languages: 92
Number of nodes: 2591
Number of edges: 6487

indo1319
Number of languages: 97
Number of nodes: 2591
Number of edges: 6546

atla1278
Number of languages: 226
Number of nodes: 2591
Number of edges: 7914

aust1307
Number of languages: 230
Number of nodes: 2591
Number of edges: 8149



## create subgraphs for each area

In [7]:
with open('../iso2area.pickle', 'rb') as handle:
    iso2area = pickle5.load(handle)

areas = {}

for key, item in iso2area.items():
    if item not in areas:
        areas[item] = [key]
    else:
        areas[item].append(key)

In [8]:
subgraph_dict_areas = {}
for code in ['South America', 'North America', 'Eurasia', 'Africa', 'Papunesia']:
    print(code)
    # the threshold 0.1 here controls the minimum number of languages for an edge
    # in the original undirected_net to be added in the language family subgraph
    # the value should be between 0 to 1
    subgraph_dict_areas[code] = net.create_subgraph_of_an_area(code, 0.1, area_path='../iso2area.pickle')
    print(f"Number of languages: {len(areas[code])}")
    print(f"Number of nodes: {subgraph_dict_areas[code].number_of_nodes()}")
    print(f"Number of edges: {subgraph_dict_areas[code].number_of_edges()}")
    print()

South America
Number of languages: 170
Number of nodes: 2591
Number of edges: 12060

North America
Number of languages: 185
Number of nodes: 2591
Number of edges: 12682

Eurasia
Number of languages: 216
Number of nodes: 2591
Number of edges: 12102

Africa
Number of languages: 352
Number of nodes: 2591
Number of edges: 13452

Papunesia
Number of languages: 396
Number of nodes: 2591
Number of edges: 13543



## Compare similarity of language family specific networks

In [9]:
from sklearn.metrics.cluster import adjusted_rand_score
import community as community_louvain
import random

In [10]:
# for each language family, we fix the oder of the nodes (stored in a list),
# and create another list where each number indicates the community
# which the correspdonding concept is in
nodes_list = list(undirected_net.nodes)

In [11]:
# add the base ConceptNet to the dictionary
subgraph_dict_language_families['base'] = undirected_net
language_families = list(subgraph_dict_language_families.keys())

In [12]:
# based on 50 random community partition, we compute the mean and variance of 
# pairwise ARI among 6 + 1 (base) language families

pairwise_ARI_families = {}

for i in range(len(language_families)):
    lf1 = language_families[i]
    net1 = subgraph_dict_language_families[lf1]
    for j in range(i, len(language_families)):
        lf2 = language_families[j]
        net2 = subgraph_dict_language_families[lf2]
        # we randomly do 50 partitions for each language family
        for i in range(50):
            # make sure the random seed is the same when computing ARI between two families each time
            seed = random.randint(0,114514)

            # community partion of net1
            partition_dict = community_louvain.best_partition(net1, resolution=0.1, random_state=seed)
            partion_1 = [partition_dict[node] for node in nodes_list]

            # community partion of net2
            partition_dict = community_louvain.best_partition(net2, resolution=0.1, random_state=seed)
            partion_2 = [partition_dict[node] for node in nodes_list]

            if f"{lf1}-{lf2}" not in pairwise_ARI_families:
                pairwise_ARI_families[f"{lf1}-{lf2}"] = [round(adjusted_rand_score(partion_1, partion_2), 2)]
            else:
                pairwise_ARI_families[f"{lf1}-{lf2}"].append(round(adjusted_rand_score(partion_1, partion_2), 2))

### Adjusted rand index

We will use adjusted rand index (ARI) to measure the similarity of the community structure between different language families. Generally speaking, the larger the ARI score, the more similar the family-specific networks are. ARI is symmetric, i.e., ARI(P1, P2) = ARI(P2, P1). But note that there will be some "isolated" concepts in these sub-networks, which can affect the ARI (making them artificially low in general).

***Negative ARI values*** indicate that two networks’ community partitions vary more than would be expected by chance,  
***ARI values of 0*** indicate that two networks’ community partitions vary at a level that would be expected at chance,   
and ***ARI values approaching 1*** reflect high agreement in community structure between two networks. 

In [13]:
ptr_string = [f"\t"] + [f"{lf[:4]}\t\t" for lf, pl in subgraph_dict_language_families.items()]
ptr_string = ''.join(ptr_string) + '\n'
for i in range(len(language_families)):
    lf1 = language_families[i]
    ptr_string += f"{lf1[:4]}\t"
    for k in range(0, i):
        ptr_string += '\t\t'
    for j in range(i, len(language_families)):
        lf2 = language_families[j]
        mean = round(np.mean(pairwise_ARI_families[f"{lf1}-{lf2}"]), 2)
        std = round(np.std(pairwise_ARI_families[f"{lf1}-{lf2}"]), 2)
        ptr_string += f"{mean}({std})\t"
    ptr_string += '\n'
print(ptr_string)

# we have the following findings:

# (1) the subnetworks of any language families are not enough resembling the ConceptNet

# (2) ARIs between indo and otom and nucl are the lowest: 0.23, 
# which indicates that these two language families are less similar to indo

# (3) ARI between atla and aust is the highest: 0.37,
# which indicates that these two language families might be similar in terms of conceptualization

# (4) ARIs between nucl with any other language families, expept aust,
# are all lower than 0.3, which indicates the conceptualization of nucl
# might be more or less unique.

	sino		otom		nucl		indo		atla		aust		base		
sino	1.0(0.0)	0.37(0.01)	0.35(0.01)	0.42(0.09)	0.47(0.03)	0.44(0.01)	0.33(0.02)	
otom			1.0(0.0)	0.4(0.01)	0.29(0.06)	0.41(0.01)	0.43(0.02)	0.3(0.02)	
nucl					1.0(0.0)	0.27(0.06)	0.38(0.01)	0.43(0.01)	0.27(0.02)	
indo							1.0(0.0)	0.37(0.08)	0.34(0.06)	0.29(0.02)	
atla									1.0(0.0)	0.49(0.02)	0.35(0.02)	
aust											1.0(0.0)	0.35(0.02)	
base													1.0(0.0)	



## Compare similarity of area specific networks

In [14]:
nodes_list = list(undirected_net.nodes)
subgraph_dict_areas['base'] = undirected_net
areas = list(subgraph_dict_areas.keys())

In [15]:
pairwise_ARI_areas = {}

for i in range(len(areas)):
    a1 = areas[i]
    net1 = subgraph_dict_areas[a1]
    for j in range(i, len(areas)):
        a2 = areas[j]
        net2 = subgraph_dict_areas[a2]
        # we randomly do 50 partitions for each language family
        for i in range(50):
            # make sure the random seed is the same when computing ARI between two families each time
            seed = random.randint(0,114514)

            # community partion of net1
            partition_dict = community_louvain.best_partition(net1, resolution=0.1, random_state=seed)
            partion_1 = [partition_dict[node] for node in nodes_list]

            # community partion of net2
            partition_dict = community_louvain.best_partition(net2, resolution=0.1, random_state=seed)
            partion_2 = [partition_dict[node] for node in nodes_list]

            if f"{a1}-{a2}" not in pairwise_ARI_areas:
                pairwise_ARI_areas[f"{a1}-{a2}"] = [round(adjusted_rand_score(partion_1, partion_2), 2)]
            else:
                pairwise_ARI_areas[f"{a1}-{a2}"].append(round(adjusted_rand_score(partion_1, partion_2), 2))

In [16]:
ptr_string = [f"\t"] + [f"{a}\t\t" for a, pl in subgraph_dict_areas.items()]
for i in range(len(ptr_string)):
    ptr_string[i] = ptr_string[i].replace('Papunesia\t\t', 'Papunesia\t')
    ptr_string[i] = ptr_string[i].replace('South America', 'SA')
    ptr_string[i] = ptr_string[i].replace('North America', 'NA')

ptr_string = ''.join(ptr_string) + '\n'
for i in range(len(areas)):
    a1 = areas[i]
    if a1 == 'South America':
        ptr_string += f"SA\t"
    elif a1 == 'North America':
        ptr_string += f"NA\t"
    else:
        ptr_string += f"{a1}\t"
    for k in range(0, i):
        ptr_string += '\t\t'
    if a1 == 'Papunesia':
        ptr_string = ptr_string[:-1]
    for j in range(i, len(areas)):
        a2 = areas[j]
        mean = round(np.mean(pairwise_ARI_areas[f"{a1}-{a2}"]), 2)
        std = round(np.std(pairwise_ARI_areas[f"{a1}-{a2}"]), 2)
        ptr_string += f"{mean}({std})\t"
    ptr_string += '\n'
print(ptr_string)

# we have the following findings:

# (1) the subnetworks of Papunesia has the highest ARI with the ConceptNet, this indicates
# that languages are the most DIVERGE in Papunesia, since ConceptNet considers associations in all languages

# (2) although SA and NA are both in America, the conceptulization seems to be diverging:
# with a mean of 0.66 ARI, relatively low compared to other pairs

# (3) Africa and Papunesia have a high ARI, this might indicates that the languages contained 
# in these two areas can have very DIVERGE conceptualizations as their ARI with ConceptNet is also high,
# with 0.85 and 0.88 respectively.

	SA		NA		Eurasia		Africa		Papunesia	base		
SA	1.0(0.0)	0.58(0.03)	0.51(0.03)	0.56(0.03)	0.56(0.02)	0.56(0.03)	
NA			1.0(0.0)	0.55(0.03)	0.61(0.04)	0.62(0.03)	0.62(0.02)	
Eurasia					1.0(0.0)	0.62(0.03)	0.6(0.03)	0.61(0.04)	
Africa							1.0(0.0)	0.76(0.04)	0.78(0.03)	
Papunesia								1.0(0.0)	0.8(0.04)	
base											1.0(0.0)	

